In [56]:
# criar um navegador
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# importar/visualizar a base de dados
pd.read_excel('buscas.xlsx')   

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,Iphone 14 128 gb,mini watch vitrine usado,3000,5500
1,rtx 3060,zota galax,4000,4500


In [92]:
def find_google(driver, product, banned_terms, min_price, max_price):
    list_google = []

    banned_terms = banned_terms.lower()
    list_banned_terms = banned_terms.split(' ')
    list_product_terms = product.split(' ')

    # Pesquisar o nome do produto no Google
    driver.get('https://www.google.com.br')
    sleep(1)
    # Localizar a caixa de pesquisa e enviar o nome do produto
    driver.find_element(By.CLASS_NAME, 'gLFyf').send_keys(product, Keys.ENTER)

    sleep(1)
    list_items = driver.find_elements(By.XPATH, '//div[@role="listitem"]')

    sleep(1)
    for item in list_items:
        if 'Shopping' in item.text:
            item.click()
            break
    sleep(1)
    # Pegar a lista de resultados da busca no Google Shopping
    list_results = WebDriverWait(driver, 2).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'i0X6df')))

    # Processa os resultados
    for result in list_results:
        try:
            # Pega o nome do produto
            name = WebDriverWait(result, 2).until(EC.presence_of_element_located((By.TAG_NAME, 'h3'))).text.lower()

            # Verifica se o nome contém termos banidos
            banned_terms_found = False
            for banned_term in list_banned_terms:
                if banned_term in name:
                    banned_terms_found = True
                    break

            # Verifica se o nome contém todos os termos do produto
            product_terms_found = True
            for product_term in list_product_terms:
                if product_term not in name:
                    product_terms_found = False
                    break

            # Se o nome não contém termos banidos e contém os termos do produto
            if not banned_terms_found and product_terms_found:
                # Pega o preço do produto
                price_element = WebDriverWait(result, 2).until(EC.presence_of_element_located((By.CLASS_NAME, 'a8Pemb')))
                price_text = price_element.text
                price = float(price_text.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.'))

                # Verifica se o preço está dentro da faixa desejada
                if min_price <= price <= max_price:
                    # Pega o link do produto
                    element_ref = result.find_element(By.CLASS_NAME, 'bONr3b')
                    elemento_pai = element_ref.find_element(By.XPATH, '..')  
                    link = elemento_pai.get_attribute('href')
                    
                    list_google.append((name, price, link))

        except Exception as e:
            print(f"Erro ao processar item: {e}")
            continue

    return list_google


def find_buscape(driver, product, banned_terms, min_price, max_price):
    list_buscape = []

    banned_terms = banned_terms.lower()
    list_banned_terms = banned_terms.split(' ')
    list_product_terms = product.split(' ')

    # Pesquisar o nome do produto no Google
    driver.get('https://www.buscape.com.br/')
    sleep(1)
    # Localizar a caixa de pesquisa e enviar o nome do produto
    driver.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(product, Keys.ENTER)

    sleep(1)
    list_items = driver.find_elements(By.CLASS_NAME, 'AutoCompleteStyle_autocomplete__BvELB')

    sleep(1)
    for item in list_items:
        if 'Shopping' in item.text:
            item.click()
            break
    sleep(1)
    # Pegar a lista de resultados da busca no Google Shopping
    list_results = WebDriverWait(driver, 2).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'Hits_ProductCard__Bonl_')))

    # Processa os resultados
    for result in list_results:
        try:
            # Pega o nome do produto
            name = WebDriverWait(result, 2).until(EC.presence_of_element_located((By.XPATH, '//*[@data-testid="product-card::name"]'))).text.lower()

            # Verifica se o nome contém termos banidos
            banned_terms_found = False
            for banned_term in list_banned_terms:
                if banned_term in name:
                    banned_terms_found = True
                    break

            # Verifica se o nome contém todos os termos do produto
            product_terms_found = True
            for product_term in list_product_terms:
                if product_term not in name:
                    product_terms_found = False
                    break

            # Se o nome não contém termos banidos e contém os termos do produto
            if not banned_terms_found and product_terms_found:
                # Pega o preço do produto
                price_element = WebDriverWait(result, 2).until(EC.presence_of_element_located((By.XPATH, '//*[@data-testid="product-card::price"]')))
                price_text = price_element.text
                price = float(price_text.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.'))

                # Verifica se o preço está dentro da faixa desejada
                if min_price <= price <= max_price:
                    # Pega o link do produto
                    link  = result.find_element(By.XPATH, '//a[@data-testid="product-card::card"]').get_attribute('href')
                    #elemento_pai = element_ref.find_element(By.XPATH, '..')  
                    #link = elemento_pai.get_attribute('href')
                    
                    list_buscape.append((name, price, link))

        except Exception as e:
            print(f"Erro ao processar item: {e}")
            continue

    return list_buscape


In [94]:
# criar o navegador
driver = webdriver.Chrome()
# tratar os valores que vieram da tabela
product = 'iphone 14 128 gb'
banned_terms = 'mmini watch vitrine usado'
min_price = 3000
max_price = 5500

# chamar a função 
list_google = find_google(driver, product, banned_terms, min_price, max_price)

list_buscape = find_buscape(driver, product, banned_terms, min_price, max_price)

driver.quit()


# Criar DataFrames de exemplo
df_google = pd.DataFrame(list_google, columns=['Nome', 'Preço', 'Link'])
df_buscape = pd.DataFrame(list_buscape, columns=['Nome', 'Preço', 'Link'])

# Combinar os DataFrames usando a coluna 'Nome' como chave
#df_complet = pd.merge(df_buscape, df_google, on='Nome', how='inner', suffixes=('_buscape', '_google'))

# Exibir o DataFrame combinado
display(df_google, df_buscape)

,Nome,Preço,Link
0,"apple iphone 14 128gb 6,1” 12mp ios 5g",3219.00,https://www.google.com.br/url?url=https://melh...
1,iphone 14 128gb - preto - tenho minhas marcas ...,3685.82,https://www.google.com.br/url?url=https://www....
2,"iphone 14 apple (128gb) meia-noite, tela de 6,...",4554.00,https://www.google.com.br/url?url=https://www....
3,apple iphone 14 plus 128gb meia-noite,4299.00,https://www.google.com.br/url?url=https://www....
4,seminovo iphone 14 128gb estelar - condição ex...,3699.00,https://www.google.com.br/url?url=https://www....
5,apple iphone 14 128gb - meia-noite,4299.00,https://www.google.com.br/url?url=https://www....
6,apple iphone 14 branco starlight - chip a15 - ...,3658.78,https://www.google.com.br/url?url=https://www....
7,apple iphone 14 (128 gb) - meia noite - excele...,3847.00,https://www.google.com.br/url?url=https://www....
8,"iphone 14 apple (128gb) preto, tela de 6,1&amp...",4499.90,https://www.google.com.br/url?url=https://www....
9,iphone 14 pro 128gb - preto - tenho minhas mar...,4789.00,https://www.google.com.br/url?url=https://www....


,Nome,Preço,Link
0,celular apple iphone 14 plus 128gb 6 gb,4139.1,https://www.buscape.com.br/celular/smartphone-...
1,celular apple iphone 14 plus 128gb 6 gb,4139.1,https://www.buscape.com.br/celular/smartphone-...
2,celular apple iphone 14 plus 128gb 6 gb,4139.1,https://www.buscape.com.br/celular/smartphone-...
3,celular apple iphone 14 plus 128gb 6 gb,4139.1,https://www.buscape.com.br/celular/smartphone-...
4,celular apple iphone 14 plus 128gb 6 gb,4139.1,https://www.buscape.com.br/celular/smartphone-...
5,celular apple iphone 14 plus 128gb 6 gb,4139.1,https://www.buscape.com.br/celular/smartphone-...
6,celular apple iphone 14 plus 128gb 6 gb,4139.1,https://www.buscape.com.br/celular/smartphone-...
7,celular apple iphone 14 plus 128gb 6 gb,4139.1,https://www.buscape.com.br/celular/smartphone-...
8,celular apple iphone 14 plus 128gb 6 gb,4139.1,https://www.buscape.com.br/celular/smartphone-...
9,celular apple iphone 14 plus 128gb 6 gb,4139.1,https://www.buscape.com.br/celular/smartphone-...
